# Import Libraries

In [1]:
# Selenium Imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# Other imports here
import os
import wget
import numpy as np
import pandas as pd
import time
import re
from IPython.display import clear_output
import pyautogui

# Initialize driver

In [3]:
# download 'Chrome Driver' and run it
driver = webdriver.Chrome('F:/downloood/chromedriver.exe')

# Declare a Dataframe to store the usernames
df = pd.DataFrame()

C:\Users\user\AppData\Local\Temp\ipykernel_4652\4188815714.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('F:/downloood/chromedriver.exe')


# Log in and close the popups

In [4]:
# Open instagram with get() function and wait for it to open
driver.get('https://www.instagram.com/')
time.sleep(5)

# Target the username and password element and input
username = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='username']")))
password = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='password']")))

# clears the field if by any chance it is autofilled
username.clear()
password.clear()

# Fills the username and pass
username.send_keys("Your-username-here")
password.send_keys("Your-Password-here")

# Press Submit button
time.sleep(2)
username = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type='submit']"))).click()

# press 'Not Now' button twice
not_now = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Not Now')]"))).click()
time.sleep(5)
driver.find_element(By.XPATH, "//button[contains(text(), 'Not Now')]").click()

# Gather usernames

In [5]:
#Input the profile link of the user whose followers and following are to be gathered
link = 'https://www.instagram.com/selenagomez/'

driver.get(link)
time.sleep(6)
driver.maximize_window()

# open the followers list
driver.find_element(By.XPATH,"//*[contains(text(), ' followers')]").click()
time.sleep(1)

#scroll
Clicks = 1000 #Set the amount to be scrolled (i.e for 500 followers 1500 clicks are recommended)
count=0
while count<Clicks:
    #MIGHT HAVE TO ADJUST THE COORDINATES FOR YOUR SCREEN SIZE
    pyautogui.click(1140, 692)
    count = count +1

#Fetch Followers
followers = []
followers = driver.find_elements(By.CSS_SELECTOR, "div > div > span > a")
followers = [follower.get_attribute('href') for follower in followers]

driver.get(link)
time.sleep(5)

# open the Following list
driver.find_element(By.XPATH,"//*[contains(text(), ' following')]").click()
time.sleep(1)

#scroll
count=0
while count<Clicks:
    pyautogui.click(1140, 710)
    count = count +1

#Fetch Following
following = []
following = driver.find_elements(By.CSS_SELECTOR, "div > div > span > a")
following = [follower.get_attribute('href') for follower in following]

print(following)
time.sleep(3)
clear_output()

#get usernames 
count = -1        
for follower in followers:
    count = count + 1
    followers[count] = follower.split("com/")[1]
    followers[count] = followers[count].split("/")[0]

count = -1        
for follower in following:
    count = count + 1
    following[count] = follower.split("com/")[1]
    following[count] = following[count].split("/")[0]

time.sleep(2)
df['Followers'] = pd.Series(followers)
df['Following'] = pd.Series(following)

In [6]:
df.describe()

,Followers,Following
count,663,227
unique,663,227
top,123456789yvra,tuna__turner
freq,1,1


# Now that we have gathered the follower list we can drive insights from it depending on the use case

In [7]:
# Get to know which accounts you follow but they dont follow you back
NotFollowingBack = []
for follower in df['Following']:
    if follower not in set(df['Followers']):
            NotFollowingBack.append(follower)
df['Not Following Back'] = pd.Series(NotFollowingBack)
df['Not Following Back']

0        tuna__turner
1        kristenlkish
2            heisrema
3      serenawilliams
4         sofiacarson
            ...      
658               NaN
659               NaN
660               NaN
661               NaN
662               NaN
Name: Not Following Back, Length: 663, dtype: object

In [8]:
df

,Followers,Following,Not Following Back
0,123456789yvra,tuna__turner,tuna__turner
1,emanuellasantana437,kristenlkish,kristenlkish
2,_mr_smarty7777,heisrema,heisrema
3,aadi_saini158,serenawilliams,serenawilliams
4,_benaivo_,sofiacarson,sofiacarson
...,...,...,...
658,nikitanazli,NaN,NaN
659,pinkukashyap848,NaN,NaN
660,viviangodoi56,NaN,NaN
661,shamaravbatakar,NaN,NaN


# Save the gathered data

In [9]:
#save the data as a .CSV file
df.to_csv('MyInstagramFollowers.csv',index=False)